In [ ]:
import yaml

# Load the config.yaml file
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [6]:
# from openai import OpenAI

# client = OpenAI(
#   api_key=config["openai_key"]

# )
import openai

openai.api_key = config['openai_key']

completion = openai.ChatCompletion.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "write a haiku about AI"}
    ]
)

resp = completion["choices"][0]["message"]["content"]

print(resp)  # Print the response


Wires hum with knowledge,  
Thoughts of code and light converge,  
Dreams in circuits bloom.


In [7]:
resp

'Wires hum with knowledge,  \nThoughts of code and light converge,  \nDreams in circuits bloom.'

Finetuning to detect arguments in a paragraph

In [25]:
import openai
import json

openai.api_key = config["openai_key"]

# Step 1: Upload the JSONL File
response = openai.File.create(
    file=open("argumentative_sentences_finetuning.jsonl", "rb"),
    purpose="fine-tune"
)

file_id = response["id"]
print(f"Uploaded file ID: {file_id}")

# Step 2: Start Fine-Tuning Job
fine_tune_response = openai.FineTuningJob.create(
    training_file=file_id,
    model="gpt-3.5-turbo"
)

job_id = fine_tune_response["id"]
print(f"Fine-tuning job started with ID: {job_id}")


Uploaded file ID: file-Q7pgopFQMATomeszS2JP8q
Fine-tuning job started with ID: ftjob-zO22PDdApQaGp01RUlGhRvyL


In [27]:
jobs = openai.FineTuningJob.list()
jobs['data']

[<FineTuningJob fine_tuning.job id=ftjob-zO22PDdApQaGp01RUlGhRvyL at 0x108cd0180> JSON: {
   "object": "fine_tuning.job",
   "id": "ftjob-zO22PDdApQaGp01RUlGhRvyL",
   "model": "gpt-3.5-turbo-0125",
   "created_at": 1740281743,
   "finished_at": null,
   "fine_tuned_model": null,
   "organization_id": "org-zygaUE62m7TS2mf85oSu07Rc",
   "result_files": [],
   "status": "running",
   "validation_file": null,
   "training_file": "file-Q7pgopFQMATomeszS2JP8q",
   "hyperparameters": {
     "n_epochs": 3,
     "batch_size": 1,
     "learning_rate_multiplier": 2
   },
   "trained_tokens": null,
   "error": {},
   "user_provided_suffix": null,
   "seed": 608759747,
   "estimated_finish": 1740282150,
   "integrations": [],
   "method": {
     "type": "supervised",
     "supervised": {
       "hyperparameters": {
         "n_epochs": 3,
         "batch_size": 1,
         "learning_rate_multiplier": 2.0
       }
     }
   }
 },
 <FineTuningJob fine_tuning.job id=ftjob-VzYzBuV57aEOwTjkc66It8GR at 

In [30]:
job_id

'ftjob-zO22PDdApQaGp01RUlGhRvyL'

In [48]:
import openai
import time
finetuned_models = []

def fine_tune_model(training_file_path, base_model="gpt-3.5-turbo"):
    """
    Fine-tune an OpenAI model with a given dataset.
    """
    # Upload training file
    response = openai.File.create(
        file=open(training_file_path, "rb"),
        purpose="fine-tune"
    )
    file_id = response["id"]
    print(f"Uploaded file ID: {file_id}")

    # Start fine-tuning
    fine_tune_response = openai.FineTuningJob.create(
        training_file=file_id,
        model=base_model
    )

    job_id = fine_tune_response["id"]
    print(f"Fine-tuning job started with ID: {job_id}")
    
    return job_id


def check_fine_tune_status(job_id):
    """
    Check the status of the fine-tuning job.
    """
    while True:
        job_status = openai.FineTuningJob.retrieve(job_id)
        status = job_status["status"]
        print(f"Fine-tuning status: {status}")
        
        if status in ["succeeded", "failed", "cancelled"]:
            print(f"Fine-tuning finished with status: {status}")
            break
        
        time.sleep(30)  # Check every 30 seconds


def get_finetuned_model_name(job_id):
    """
    Retrieves the fine-tuned model name using the fine-tuning job ID.
    """
    job_info = openai.FineTuningJob.retrieve(job_id)
    fine_tuned_model = job_info.get("fine_tuned_model")

    if fine_tuned_model:
        print(f"Fine-Tuned Model Name: {fine_tuned_model}")
        return fine_tuned_model
    else:
        print("Fine-tuning is still in progress or has failed.")
        return None

In [21]:
# to cancel any specific finetuning job
# cancel_response = openai.FineTuningJob.cancel(job_id)
# cancel_response

In [31]:
# Retrieve fine-tuning job details
job_info = openai.FineTuningJob.retrieve("ftjob-zO22PDdApQaGp01RUlGhRvyL")

# Get the fine-tuned model ID
fine_tuned_model = job_info.get("fine_tuned_model")

if fine_tuned_model:
    print(f"Your fine-tuned model ID is: {fine_tuned_model}")
else:
    print("Fine-tuning job is still in progress or failed.")

Your fine-tuned model ID is: ft:gpt-3.5-turbo-0125:personal::B3xAsgP1


In [40]:
argument_detection_model = fine_tuned_model

def detect_argument(statement):
    response = openai.ChatCompletion.create(
        model=fine_tuned_model,  # Replace with your fine-tuned model ID
        messages=[
            {"role": "system", "content": "You are a critical analysis AI that evaluates statements for argument strength."},
            {"role": "user", "content": f"Statement: '{statement}'\nDoes the statement make any claims, chat with the user to encourage them to ask some critical questions (with potential line of thoughts to have) before forming a conclusion from the statement"}
        ]
    )

    print(response["choices"][0]["message"]["content"])


In [41]:
statement = "A Fargo, North Dakota, man was arrested for clearing snow with a flamethrower."

detect_argument(statement)

This statement lacks a clear stance or reasoning. Before forming a conclusion, it would be helpful to ask: What are the potential implications or reasoning behind the man's actions? Is there any context provided or reasoning given for his behavior?


Finetuning atomizer

In [42]:

atomizer_job_id = fine_tune_model("atomization_critical_analysis_finetuning_50.jsonl")
check_fine_tune_status(atomizer_job_id)

Uploaded file ID: file-3kgciCTKZWzLX3GrQ8hxzF
Fine-tuning job started with ID: ftjob-lN0ZXS21LfRw1QhFXkO92Ndu
Fine-tuning status: validating_files
Fine-tuning status: validating_files
Fine-tuning status: validating_files
Fine-tuning status: running
Fine-tuning status: running
Fine-tuning status: running
Fine-tuning status: running
Fine-tuning status: running
Fine-tuning status: running
Fine-tuning status: succeeded
Fine-tuning finished with status: succeeded


In [43]:
finetuned_models = []
job_status = openai.FineTuningJob.retrieve(atomizer_job_id)
print(f"Fine-tuning status: {job_status['status']}")

if job_status == 'succeeded':
    atomizer_model_name = get_finetuned_model_name(atomizer_job_id)

    finetuned_models.append({'atomizer_model_name': atomizer_model_name})

Fine-tuning status: succeeded


In [44]:
def get_fine_tuned_model():
    """
    Retrieve the fine-tuned model ID.
    """
    fine_tuned_models = openai.FineTuningJob.list()
    for job in fine_tuned_models["data"]:
        if job["status"] == "succeeded":
            print(f"Fine-tuned Model ID: {job['fine_tuned_model']}")
            return job['fine_tuned_model']
    
    print("No fine-tuned model found.")
    return None

# Example Usage:
fine_tuned_model_id = get_fine_tuned_model()
fine_tuned_model_id

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:personal::B3yyOMLV


'ft:gpt-3.5-turbo-0125:personal::B3yyOMLV'

In [52]:
argument_finetuned_model = get_finetuned_model_name('ftjob-zO22PDdApQaGp01RUlGhRvyL')

Fine-Tuned Model Name: ft:gpt-3.5-turbo-0125:personal::B3xAsgP1


In [53]:
atomizer_finetuned_model = get_finetuned_model_name(atomizer_job_id)

Fine-Tuned Model Name: ft:gpt-3.5-turbo-0125:personal::B3yyOMLV


In [ ]:
import time

def check_fine_tune_status(job_id):
    while True:
        job_status = openai.FineTuningJob.retrieve(job_id)
        status = job_status["status"]
        print(f"Fine-tuning status: {status}")
        
        if status in ["succeeded", "failed", "cancelled"]:
            print(f"Fine-tuning finished with status: {status}")
            break
        
        time.sleep(30)  # Check every 30 seconds

# Replace with your actual job ID
check_fine_tune_status(job_id)


In [54]:
finetuned_models = []

finetuned_models.append({'argument_finetuned_model': argument_finetuned_model})
finetuned_models.append({'atomizer_finetuned_model': atomizer_finetuned_model})

In [57]:
critical_fine_tuned_model_id = fine_tune_model("critical_analysis_finetuning_50.jsonl")
check_fine_tune_status(critical_fine_tuned_model_id)

Uploaded file ID: file-Y5qXqRKZyiNRteESBTWiT8
Fine-tuning job started with ID: ftjob-AqtIXTDsomBgtA02llDOF5gL
Fine-tuning status: validating_files
Fine-tuning status: validating_files
Fine-tuning status: validating_files
Fine-tuning status: running
Fine-tuning status: running
Fine-tuning status: running
Fine-tuning status: running
Fine-tuning status: running
Fine-tuning status: running
Fine-tuning status: running
Fine-tuning status: running
Fine-tuning status: running
Fine-tuning status: running
Fine-tuning status: running
Fine-tuning status: succeeded
Fine-tuning finished with status: succeeded


In [58]:
critical_fine_tuned_model = get_finetuned_model_name(critical_fine_tuned_model_id)
finetuned_models.append({'critical_fine_tuned_model': critical_fine_tuned_model})

Fine-Tuned Model Name: ft:gpt-3.5-turbo-0125:personal::B3zr6iGR


In [59]:
finetuned_models

[{'argument_finetuned_model': 'ft:gpt-3.5-turbo-0125:personal::B3xAsgP1'},
 {'atomizer_finetuned_model': 'ft:gpt-3.5-turbo-0125:personal::B3yyOMLV'},
 {'critical_fine_tuned_model': 'ft:gpt-3.5-turbo-0125:personal::B3zr6iGR'}]

automating the end to end processing of input para to generate critical line of thoughts, to explore

In [61]:
finetuned_models

[{'argument_finetuned_model': 'ft:gpt-3.5-turbo-0125:personal::B3xAsgP1'},
 {'atomizer_finetuned_model': 'ft:gpt-3.5-turbo-0125:personal::B3yyOMLV'},
 {'critical_fine_tuned_model': 'ft:gpt-3.5-turbo-0125:personal::B3zr6iGR'}]

In [63]:
import openai

openai.api_key = config['openai_key']

# Fine-tuned model names
argument_finetuned_model = finetuned_models[0]['argument_finetuned_model']
atomizer_finetuned_model = finetuned_models[1]['atomizer_finetuned_model']
critical_fine_tuned_model = finetuned_models[2]['critical_fine_tuned_model']

def analyze_paragraph(paragraph):
    """
    End-to-end function that:
    1. Detects if an argument/strong stance exists.
    2. Extracts claims and atomizes them.
    3. Generates critical perspectives for each atomic claim.
    Returns structured critical thoughts.
    """

    ### 🔹 Step 1: Detect Argument in Paragraph ###
    argument_detection_prompt = f"""
    Analyze the given paragraph and determine whether it contains a strong argument, claim, or stance.
    If the paragraph contains an argument, extract the key claims or positions stated.
    Output should be either:
    None - if no arguments found in paragraph, if present, return the lines in the paragraph that contains the arguments. 
    Use the paragraph below to infer the arguments if any.
    Paragraph: {paragraph}
    """
    
    response = openai.ChatCompletion.create(
        model=argument_finetuned_model,
        messages=[{"role": "system", "content": "You are an AI that detects arguments and extracts claims from text."},
                  {"role": "user", "content": argument_detection_prompt}]
    )
    
    argument_output = response["choices"][0]["message"]["content"]

    if "No Argument Found" in argument_output:
        return {"message": "No argument detected in the paragraph."}

    extracted_claims = argument_output.split("\n")  # Assuming claims are line-separated

    ### 🔹 Step 2: Atomize the Extracted Claims ###
    atomized_claims = []
    
    for claim in extracted_claims:
        atomization_prompt = f"""
        Break down the following claim into atomic facts.
        Ensure each fact is self-contained, concise, and retains the original meaning.
        Output should be a structured list of atomic claims.

        Claim: {claim}
        """
        
        response = openai.ChatCompletion.create(
            model=atomizer_finetuned_model,
            messages=[{"role": "system", "content": "You are an AI that breaks down complex claims into atomic claims."},
                      {"role": "user", "content": atomization_prompt}]
        )
        
        atomized_output = response["choices"][0]["message"]["content"]
        atomized_claims.extend(atomized_output.split("\n"))  # Collecting all atomic claims

    ### 🔹 Step 3: Generate Critical Perspectives ###
    critical_thoughts = []

    for atomic_claim in atomized_claims:
        critique_prompt = f"""
        Critically analyze the following atomic claim.
        Provide alternative perspectives, question underlying assumptions, and explore potential biases.
        Generate a structured critical assessment.

        Atomic Claim: {atomic_claim}
        """
        
        response = openai.ChatCompletion.create(
            model=critical_fine_tuned_model,
            messages=[{"role": "system", "content": "You are an AI that generates critical perspectives on claims."},
                      {"role": "user", "content": critique_prompt}]
        )
        
        critique_output = response["choices"][0]["message"]["content"]
        critical_thoughts.append({"claim": atomic_claim, "critique": critique_output})

    return {"original_text": paragraph, "critical_analysis": critical_thoughts}

# Example Usage:
paragraph = "Artificial intelligence will replace most jobs, but it will also create new opportunities in various sectors."
result = analyze_paragraph(paragraph)
print(result)


{'original_text': 'Artificial intelligence will replace most jobs, but it will also create new opportunities in various sectors.', 'critical_analysis': [{'claim': '- A structured list of extracted claims', 'critique': 'Critical Assessment: The provided atomic claim - A structured list of extracted claims assumes This house would defund the police functions in a certain way, but it is essential to consider alternative perspectives. While A structured list of extracted claims, opposing research suggests that This house would defund the police could have different impacts under varying conditions. This house would defund the police-specific policies might be context-dependent, influenced by factors like A structured list of extracted claims-specific socioeconomic backgrounds, This house would defund the police This house would defund the police, or external variables. Therefore, this critical assessment highlights the need to consider a diverse range of factors when evaluating The provide

In [76]:
finetuned_models = {'argument_finetuned_model': 'ft:gpt-3.5-turbo-0125:personal::B3xAsgP1', 
                    'atomizer_finetuned_model': 'ft:gpt-3.5-turbo-0125:personal::B3yyOMLV',
                    'critical_fine_tuned_model': 'ft:gpt-3.5-turbo-0125:personal::B3zr6iGR'}

[{'argument_finetuned_model': 'ft:gpt-3.5-turbo-0125:personal::B3xAsgP1'},
 {'atomizer_finetuned_model': 'ft:gpt-3.5-turbo-0125:personal::B3yyOMLV'},
 {'critical_fine_tuned_model': 'ft:gpt-3.5-turbo-0125:personal::B3zr6iGR'}]

In [75]:
import openai

openai.api_key = config['openai_key']

# Fine-tuned model names
argument_finetuned_model = finetuned_models[0]['argument_finetuned_model']
atomizer_finetuned_model = finetuned_models[1]['atomizer_finetuned_model']
critical_fine_tuned_model = finetuned_models[2]['critical_fine_tuned_model']

def analyze_paragraph(paragraph):
    """
    End-to-end function that:
    1. Detects if an argument/strong stance exists.
    2. Extracts claims and atomizes them.
    3. Generates critical perspectives for each atomic claim.
    Returns structured critical thoughts.
    """

    # ### 🔹 Step 1: Detect Argument in Paragraph ###
    # argument_detection_prompt = f"""
    # Analyze the given paragraph and determine whether it contains a strong argument, claim, or stance.
    # If the paragraph contains an argument, extract the key claims or positions stated.
    # Output should be either:
    # if no arguments found in paragraph - return None
    # else, only return the lines in the paragraph that contains the arguments. 
    # For example,
    # Input: 'So if you think that people get value off of art or if people get value from listening to music, if it makes our lives better it's important, make sure the you're supporting the people who give you that sense of happiness when you listen to a great song and debate we which this is supported, is by making sure they're able to make a living by doing what they're doing, but without an intellectual property right it's, impossible for them to actually make this living because anyone can just rip them off use their song'
    # Output: 'It argues why we should not abolish intellectual property rights'"

    # Using the above as an example, now infer the arguments if any present in the paragraph given by user.
    # """
    
    # response = openai.ChatCompletion.create(
    #     model=argument_finetuned_model,
    #     messages=[{"role": "system", "content": argument_detection_prompt},
    #               {"role": "user", "content": paragraph}]
    # )
    
    # argument_output = response["choices"][0]["message"]["content"]

    # # if "No Argument Found" in argument_output:
    # #     return {"message": "No argument detected in the paragraph."}

    # extracted_claims = argument_output.split("\n")  # Assuming claims are line-separated
    extracted_claims = paragraph.split('\n')

    ### 🔹 Step 2: Atomize the Extracted Claims ###
    atomized_claims = []
    # print(f"argument model returns - {extracted_claims=}")
    for claim in extracted_claims:
        atomization_prompt = f"""
        Break down the following claim into atomic facts.
        Ensure each fact is self-contained, concise, and retains the original meaning.
        Output should be a structured list of atomic claims.

        Claim: {claim}
        """
        
        response = openai.ChatCompletion.create(
            model=atomizer_finetuned_model,
            messages=[{"role": "system", "content": "You are an AI that breaks down complex claims into atomic claims."},
                      {"role": "user", "content": atomization_prompt}]
        )
        
        atomized_output = response["choices"][0]["message"]["content"]
        atomized_claims.extend(atomized_output.split("\n"))  # Collecting all atomic claims

    ### 🔹 Step 3: Generate Critical Perspectives ###
    critical_thoughts = []

    for atomic_claim in atomized_claims:
        critique_prompt = f"""
        Critically analyze the following atomic claim.
        Provide alternative perspectives, question underlying assumptions, and explore potential biases.
        Generate a structured critical assessment.

        Atomic Claim: {atomic_claim}
        """
        
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=[{"role": "system", "content": """Generate critical perspectives on claims given by user, challenge and expand claims by providing thought-provoking critiques. 
                       for example, 
                       Input: 'The one-child policy has been a major success, reducing overpopulation and improving China's economy. While controversial, most citizens support it, and studies show it has led to better healthcare for women. The government remains firm in its decision, arguing that any negative consequences are outweighed by long-term benefits.'
                       Output: 'What are the ethical concerns about limiting family size? Is economic gain a justification for restricting personal freedoms?\n- How reliable are the studies supporting this policy? Are they government-funded, and do they account for hidden consequences like gender imbalance?\n- Are there long-term economic risks, such as an aging population and labor shortages, that counterbalance the short-term benefits?\n- The claim that 'most citizens support it'\u2014does this account for people who are afraid to speak against government policies?\n- What alternative policies could have achieved the same outcome with less human rights impact?'
                       Similar to above critical line of thought to evaluate the presented argument, generate critical lines of thought for user given input. """},
                      {"role": "user", "content": critique_prompt}]
        )
        
        critique_output = response["choices"][0]["message"]["content"]
        critical_thoughts.append({"claim": atomic_claim, "critique": critique_output})
    critique = '\n'.join(ct['critique'] for ct in critical_thoughts)
    critique = critique.strip(' ').strip('-').strip('\n').strip(',')
    # return {"original_text": paragraph, "critical_analysis": critical_thoughts}
    return critique


In [ ]:
# Example Usage:
paragraph = "Artificial intelligence will replace most jobs, but it will also create new opportunities in various sectors."
result = analyze_paragraph(paragraph)
print(result)   

from fastapi import FastAPI, HTTPException
app = FastAPI()

@app.post("/analyze")
async def analyze_content(content: str):
    """
    API Endpoint to analyze a webpage's parsed content.
    Expects a raw text string as input.
    Returns critical thoughts as a response.
    """
    if not content:
        raise HTTPException(status_code=400, detail="Content cannot be empty.")

    critical_analysis = analyze_paragraph(content)
    
    return {"critical_thoughts": critical_analysis}

In [ ]:
import re

def atomize_claims(paragraph):
    """
    Breaks down a complex paragraph into atomic claims.
    """
    # Splitting at conjunctions that indicate multiple ideas
    atomic_claims = [c.strip() for c in re.split(r'\band\b|\bbut\b|\bhowever\b|\bor\b|\bso\b', paragraph) if c]
    
    return atomic_claims

# Example Usage:
text = "The one-child policy helped reduce overpopulation, but it led to severe social issues like gender imbalance and aging population."
atomic_claims = atomize_claims(text)
print(atomic_claims)


import re

def atomize_claims(paragraph):
    """
    Breaks down a complex paragraph into atomic claims.
    """
    # Splitting at conjunctions that indicate multiple ideas
    atomic_claims = [c.strip() for c in re.split(r'\band\b|\bbut\b|\bhowever\b|\bor\b|\bso\b', paragraph) if c]
    
    return atomic_claims

# Example Usage:
text = "The one-child policy helped reduce overpopulation, but it led to severe social issues like gender imbalance and aging population."
atomic_claims = atomize_claims(text)
print(atomic_claims)


def set_atomizer_prompt(text):
    """
    Generates the atomizer prompt for breaking down a complex statement.
    """
    atomizer_prompt = """
    Break down the given statement into atomic facts. 
    Ensure each fact is self-contained and retains its logical meaning.
    Output a structured list of independent claims.
    """
    return f"{atomizer_prompt}\n\nInput: {text}\nOutput:"

# Example Usage:
prompt = set_atomizer_prompt(text)
print(prompt)

def generate_critical_perspective(atomic_claims):
    """
    Uses the fine-tuned critique model to generate critical perspectives for each atomic claim.
    """
    responses = []

    for claim in atomic_claims:
        response = openai.ChatCompletion.create(
            model=critical_fine_tuned_model_id,
            messages=[
                {"role": "system", "content": "You are a critical thinking AI that evaluates claims and provides critiques."},
                {"role": "user", "content": f"Analyze the following claim and provide a critical perspective: '{claim}'"}
            ]
        )
        critique = response["choices"][0]["message"]["content"]
        responses.append({"claim": claim, "critique": critique})
    
    return responses

# Example Usage:
if critical_fine_tuned_model_id:
    critiques = generate_critical_perspective(atomic_claims, fine_tuned_model_id)
    print(critiques)

def analyze_text(text, fine_tuned_model_id):
    """
    Complete workflow: 
    1. Atomize a paragraph into independent claims.
    2. Generate a critique for each atomic claim.
    """
    atomic_claims = atomize_claims(text)
    critiques = generate_critical_perspective(atomic_claims)

    return {"original_text": text, "analysis": critiques}

# Example Usage:
text = "AI will completely replace human jobs in the future, but it will also create new opportunities in different sectors."
result = analyze_text(text)
print(result)


In [56]:
# Generate a critique using the fine-tuned model
response = openai.ChatCompletion(
    model=argument_finetuned_model,
    prompt="Check for arguments made in the given text (if any): 'The earth is flat says the team of scientists'",
    max_tokens=150
)
print(response.choices[0].text.strip())

AttributeError: choices